# Data Mining in Action, Deep Leaning

Авторы ноутбука: 
1. Петр Иванов (telegram: @dataisdata, linkedin: www.linkedin.com/in/petr-ivanov-58995570/)
2. Kjetil Åmdal-Sævik

<h1>Разделы<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Семантическая-сегментация" data-toc-modified-id="Семантическая-сегментация-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Семантическая сегментация</a></span></li><li><span><a href="#Импорты" data-toc-modified-id="Импорты-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Импорты</a></span></li><li><span><a href="#Вспомогательные-функции" data-toc-modified-id="Вспомогательные-функции-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вспомогательные функции</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Создадим-метрику-для-оценки-качества-модели" data-toc-modified-id="Создадим-метрику-для-оценки-качества-модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Создадим метрику для оценки качества модели</a></span></li><li><span><a href="#Построение-сети" data-toc-modified-id="Построение-сети-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Построение сети</a></span></li><li><span><a href="#Построим-пару-графиков" data-toc-modified-id="Построим-пару-графиков-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Построим пару графиков</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Зависимость-лосса-от-количества-эпох:" data-toc-modified-id="Зависимость-лосса-от-количества-эпох:-7.0.1"><span class="toc-item-num">7.0.1&nbsp;&nbsp;</span>Зависимость лосса от количества эпох:</a></span></li><li><span><a href="#Зависимость-метрики-mean-IoU-от-количества-эпох" data-toc-modified-id="Зависимость-метрики-mean-IoU-от-количества-эпох-7.0.2"><span class="toc-item-num">7.0.2&nbsp;&nbsp;</span>Зависимость метрики mean IoU от количества эпох</a></span></li></ul></li></ul></li><li><span><a href="#Предсказания-модели" data-toc-modified-id="Предсказания-модели-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Предсказания модели</a></span></li><li><span><a href="#Подготовим-результаты-для-сабмита" data-toc-modified-id="Подготовим-результаты-для-сабмита-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Подготовим результаты для сабмита</a></span></li></ul></div>

## Семантическая сегментация

В этом задании мы будем практиковаться в семантической сегментации. Реализуем популярную сеть для решения задачи семантической сегментации U-net, обучим её на датасете с фигурами Lego, оценим качество модели.

Перед тем как приступать к выполнению задания не забудьте:
1. Настроить виртуальное окружение, инструкция находится в репозитории курса.
2. Выполнить команду ```pip install -r <path_to_rep_requirements.txt>``` для того, чтобы в вашем виртуальном окружении были установлены правильные версии python пакетов.
3. Скачать датасет в директорию, в которой находится данный ноутбук (../DMIA_DL_2019_Autumn/homework_segmentation). Датасет доступен по ссылке https://drive.google.com/open?id=1czbfhH7uMbyfM1su1uzGNU1VdoLioE43

## Импорты

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 12) # установим стандратный размер графиков

In [ ]:
import cv2
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Установим параметры изображений
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3

# Определим пути к изображениям и их маскам
TRAIN_PATH = './lego_train_images/'
TEST_PATH = './lego_test_images/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

# сделаем результаты воспроизводимыми
seed = 42
random.seed = seed
np.random.seed = seed

In [ ]:
# Получим тренировочные и тестовые ids
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

## Вспомогательные функции

In [ ]:
def hex_2_rgb(hex):
    return tuple(int(hex[i:i+2], 16) for i in (0, 2, 4))

In [ ]:
classes_pallete = ['2F8E3C','87CF6E','AC75D1','B4A83C'
                  ,'3BB0A4','DFDC9F','3D47B8','862D6D'
                  ,'88CED7','CF716E','82622B','503399'
                  ,'DDC9ED','C1448F','5381C6','FFDF64',
                  '14DB8F']

In [ ]:
class_color = [hex_2_rgb(color) for color in classes_pallete]

In [ ]:
def get_masked_image(image, mask):
    out = image.copy()
    for i in range(3):
        out[:,:,i][mask[:,:,0]==20] = 0 
    return out

In [ ]:
def get_color(x):
    return class_color[x]

get_color_v = np.vectorize(get_color)
def get_mask_from_multiclass(mask, classes = 17):
    cl = np.argmax(mask,axis=2)
    new_mask = np.transpose(np.array(get_color_v(cl)),axes=(1,2,0)).astype(np.uint8)
    return new_mask

In [ ]:
def show_ix(ix):
    image = resize(X_train[ix], (480, 640), mode='constant', preserve_range=True).astype(np.float32)
   
    mask = resize(get_mask_from_multiclass(Y_train[ix]), (480, 640), mode='constant', preserve_range=True)
    mask_pred = resize(get_mask_from_multiclass(preds_train[ix]), (480, 640), mode='constant', preserve_range=True).astype(np.float32)

    print(mask.shape)
    masked = get_masked_image(image,mask_pred)
    
    pos, axarr = plt.subplots(2, 2)
    
    axarr[0, 0].set_title('image')
    axarr[0, 0].imshow(image.astype(np.uint8))
    axarr[0, 1].set_title('image mask')
    axarr[0, 1].imshow(mask.astype(np.uint8))
    axarr[1, 0].set_title('image masked by pred')
    axarr[1, 0].imshow(masked.astype(np.uint8))
    axarr[1, 1].set_title('predicted mask')
    axarr[1, 1].imshow(mask_pred.astype(np.uint8))

## Предобработка данных


In [ ]:
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 17), dtype=np.bool)

train_image_id = []
test_image_id = []

print('Предобработка тренировочных данных ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    train_image_id.append(id_)
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        
        object_class = int(mask_file[-6:-4])
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        Y_train[n][:,:,object_class] = np.squeeze(mask_)

    Y_train[n][:,:,16] = np.logical_not(np.max(Y_train[n][:,:,:16], axis=2))

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []

print('Предобработка тестовых данных ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    test_image_id.append(id_)
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

Посмотрим как выглядят исходная картинка и ее маска:

In [ ]:
picture_index = random.randint(0, len(train_ids))
imshow(X_train[picture_index])
plt.show()
imshow(get_mask_from_multiclass(Y_train[picture_index]))
plt.show()

## Создадим метрику для оценки качества модели

Будем пользоваться индексом Жакара, усредненным по всем классам и батчу. Подробнее о ней вы можете прочитать здесь: https://en.m.wikipedia.org/wiki/Jaccard_index

<img src='https://www.pyimagesearch.com/wp-content/uploads/2016/09/iou_equation.png'>

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.cast(y_pred > t, tf.int32)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 16)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

## Построение сети

Теперь можно начать построение сети. За основу будем использовать архитектуру U-net.

<img src='https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png'>

Архитектура состоит из блоков (горизонтальные участки из синих прямоугольников). На примере выше сначала идёт 5 блоков кодировщика, далее 4 блока декодировщика. Между блоками кодировщика происходит уменьшение размерности в 2 раза с помощью max pooling. В декодировщике применяется upsampling и конкатенация с результатами соответствующего блока кодировщика. 

Давайте реализуем U-net по частям. Начнём с кодировщика. 

Один блок кодировщика состоит из следующих слоев: 
1. свертка
2. активация (будем использовать ELU)
3. дропаута
4. свертка
5. активация 
6. maxpooling

In [ ]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

In [ ]:
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)

p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)

p2 = MaxPooling2D((2, 2)) (c2)

Задание:
Реализуйте остальные 3 блока кодировщика по такому же принципу. Количество фильтров должно увеличиваться в 2 раза. 
Будьте внимательны, в последнем блоке не нужен MaxPooling.

In [ ]:
<your code>

Теперь реализуем декодировщик.
Один блок декодировщика состоит последовательно из следующих слоев: 
1. транспонированная свертка
2. конкатенация
3. свертка
4. активация
5. дропаут
6. свертка
7. активация

In [ ]:
u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

Задание: Реализуйте остальные 2 блока кодировщика по такому же принципу. Количество фильтров должно уменьшаться в 2 раза. 

In [ ]:
<your code>

На выходе из сети получаем бинарную маску из линейных активаций. Для обучения такой модели используются функции потерь, которые включают в себя применение активации. В вероятности их можно превращать с помощью sigmoid.

In [ ]:
outputs = Conv2D(17, (1, 1), activation='sigmoid') (c9)

Для обучения модели необходимо определить функцию потерь и оптимизатор. Сейчас для задачи бинарной сегментации нам потребуется бинарная кросс энтропия, которая на вход будет принимать вероятности. В качестве оптимизатора будем использовать Adam.

In [ ]:
model = Model(inputs=[inputs], outputs=[outputs])

model.compile(<задайте функцию потерь, оптимизатор и метрику IoU, реализованную выше>)

Посмотрим описание нашей модели

In [ ]:
model.summary()

Запустим тренировку модели:

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=0)
checkpointer = ModelCheckpoint('model-segmentation-homework-1.h5', verbose=0, save_best_only=True)

results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=60, 
                    callbacks=[checkpointer])

## Построим пару графиков

#### Зависимость лосса от количества эпох:

In [ ]:
history = results

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#### Зависимость метрики mean IoU от количества эпох

Задание: Постройте зависимость метрики mean IoU от количества эпох

In [ ]:
<your code>

## Предсказания модели

Давайте сделаем предсказания на тестовых, валидационных и тренировочных данных (в качестве проверки работоспособности).

In [ ]:
model = load_model('model-segmentation-homework-1.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)


preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)


preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

Посмотрим на результат предсказания

In [ ]:
picture_index = random.randint(0, len(preds_train_t))
show_ix(picture_index)

## Подготовим результаты для сабмита

In [ ]:
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

        
def bool_to_rles(x):
    lab_img = label(x)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)    
           
def join_nums(x):
    s = ''
    for z in x:
        s = s + ' '.join(str(y) for y in z) + ' '
    return s[:-1]

def rle2mask(mask_rle, shape):
    """
    mask_rle: run-length as string formated (start length)ы
    shape: (width,height) of array to return
    Returns numpy array, 1 - mask, 0 - background
    """
    if mask_rle != mask_rle:
        return np.zeros_like(shape)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
img_labels = []
cl_labels = []
encoded = []

flush_classes = np.argmax(preds_test_upsampled, axis = 3)

for i,id_ in enumerate(test_image_id):
    for j in range(16):
        rle = list(bool_to_rles(flush_classes[i] == j))
        if len(rle) > 0:
            img_labels.append(id_+'.png')
            cl_labels.append(j)
            encoded.append(rle)

In [ ]:
submission = pd.DataFrame()
submission['ImageId'] = img_labels
submission['ClassId'] = cl_labels
submission['EncodedPixels'] = pd.Series(encoded).apply(join_nums)

In [ ]:
submission.to_csv('submission.csv',index = False)